In [9]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import gradio as gr
from PIL import Image
"""This file allows the user to create a sharable UI or run one locally on
    their computer. Simply load the trained model of your choice at the bottom
    """



#This function takes in a picture as input, preforms the required transformations
#and matches the str() tensor output to an elif block to make a prediction.
def predict(inp):

    data_transforms = transforms.Compose([
           transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    
    inp = data_transforms(inp).unsqueeze(0)
    outputs = vgg16(inp)
    x, preds = torch.max(outputs.data, 1)
    labels = [preds[j] for j in range(inp.size()[0])]
    predicted_labels = str(labels)
    ans = ""
        
    if(predicted_labels == "[tensor(1)]"):
        ans = "Line graph"
    elif(predicted_labels == "[tensor(2)]"):
        ans = "Pie graph"
    elif(predicted_labels == "[tensor(0)]"):
        ans = "Bar graph"
    else:
        ans = "Unable to identify graph make sure it is either:" \
              "Bar, line or pie graph"
    
    return ans


def main():
  pass

if __name__ == '__main__':

    # # Load the pretrained model from pytorch
    vgg16 = models.vgg16_bn()

    # Freeze training for all layers
    for param in vgg16.features.parameters():
        param.require_grad = False

    # Newly created modules have require_grad=True by default
    num_features = vgg16.classifier[6].in_features
    features = list(vgg16.classifier.children())[:-1]  # Remove last layer
    features.extend([nn.Linear(num_features, 3)])  # Add our layer with 3 outputs
    vgg16.classifier = nn.Sequential(*features)  # Replace the model classifier


    #used in transfer learning
    criterion = nn.CrossEntropyLoss()

    #optimization (possibly could use atom)
    optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


    #load your trained model here
    #If your device is capable of running CUDA go ahead and delete/change the map location to reflect that
    vgg16.load_state_dict(torch.load(r"/Users/matthew/PycharmProjects/CSCI338_Project/VGG16_graphs.pt",map_location=torch.device('cpu')))


    #lock the paramters as to not overtrain
    vgg16.eval()


    #call the Gradio interface which asks
    # fn =  What function do you want me to pass input to?
    # inputs = What type of input can I accept?
    # outputs = What type of output would you like?
    gr.Interface(fn=predict,
             inputs=gr.inputs.Image(type="pil"),
             outputs=gr.outputs.Label(num_top_classes=3)).launch()

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.
